Data proessing functions

In [42]:
# Name Process - Emma@ Johnson Capital12 -> emma johnson capital12

import re

def preprocess_name(name):
    #convert to lower case
    name = name.lower()
    #remove non alphanumeric characters
    name = re.sub('[^A-Za-z0-9]+', ' ', name)
    #remove spaces from beginning and end of the string
    name = name.strip()
    #convert consecutive spaces/tabs/newlines in a single space
    name = " ".join(name.split())
    return name

In [38]:
# Process company names - HSBC & Holdings 12 ltd. -> hsbc 12

import re

def company_processor(name):
    #convert to lower case
    name = name.lower()
    #remove spaces from beginning and end of the string
    name = name.strip()
    #convert consecutive spaces/tabs/newlines in a single space
    name = " ".join(name.split())
    #convert limited (with any combination of .) to ltd and company/co. (with any combination of .) to co
    name = re.sub(r'limited', r'ltd', name)
    name = re.sub(r'l\.*t\.*d\.*', r'ltd', name)
    name = re.sub(r'company', r'co', name)
    name = re.sub(r'c\.*o.\*', r'co', name)
    #Countries
    #convert & to "and"
    name = name.replace('&', ' and ')

    text = name 

    #convert to lower case
    text = text.lower()
    
    stop_words = ['ltd', 'limited', 'holdings' ,'plc', 'of', "sme", 'and']
   #"co", "-"
   
    # Split the string into a list of words 
    words = text.split() 

    # Create a new list to hold the filtered words 
    filtered_words =  "" 

    # Iterate over the list of words 
    for word in words: 
        # If the word is not in the stop word list, add it to the filtered list 
        if word not in stop_words: 
            filtered_words = filtered_words + " " + word
    filtered_words = filtered_words.strip()       
    return filtered_words

In [39]:
# Retrieve Long & Lat of given location through API and find distance between them

import geopy as geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(timeout=10, user_agent="Krukarius")

def find_location(place):
    location = geolocator.geocode(place)
    location_overall = geolocator.geocode(location)
    return location_overall.latitude, location_overall.longitude

import haversine as hs

def Distance(loc1,loc2):
    return hs.haversine(loc1,loc2)

In [41]:
# Retrieve username from email address, and splits if there is a .

def extract_username(email):
    keywordsPhrase = ""

    if pd.isnull(email):
        keywordsPhrase = ""
    else:
        # Remove any leading or trailing spaces
        email = email.strip()

        # Split the email address into two parts: the local part and the domain part
        local_part, domain_part = email.split('@')

        # Remove any dots from the local part
        local_part = local_part.replace('.', '_')
        local_part = re.sub(r'\d+', '_', local_part)

        # Extract the keywords from the local part
        keywords = local_part.split('_')

        keywordsList = list(filter(lambda x: x.strip() != "", keywords))
        for string in keywordsList:
            keywordsPhrase += f"{string} "

    return keywordsPhrase

First-Layer Matching Process using tf-idf

In [21]:
# Creates a CSR Matrix. An efficient representation of sparse (many zeros) matrix*

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, triu
import sparse_dot_topn.sparse_dot_topn as ct

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # CSR matrix -- efficient representation of sparse (many zeros) matrix
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    # rows of A
    M, _ = A.shape
    # columns of B
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

# Efficient dot product and selection of ntop most-similar vectors
    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

In [22]:
# Get matches given matrix and dataframe

import numpy as np
import pandas as pd

def get_matches_SF_brokers(sparse_matrix, df):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    nr_matches = sparsecols.size
    
    left_row_ids = np.empty([nr_matches], dtype=object)
    right_row_ids = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    row_ids = df['ID'].values
    
    for i in range(0, nr_matches):
        left_row_ids[i] = row_ids[sparserows[i]]
        right_row_ids[i] = row_ids[sparsecols[i]]
        similairity[i] = sparse_matrix.data[i]
    
    
    return pd.DataFrame({'ID': left_row_ids,
                         'SIMILARITY': similairity,
                         'MATCHED_ID': right_row_ids})

Import Data

The data is from https://www.briandunning.com/sample-data/.
I have removed some fields in some rows in order to test accuracy of this program when we have incomplete data.

In [24]:
# Import Data

import pandas as pd

Data = pd.read_csv('SampleData.csv')

Data Wrangling

In [25]:
# address_generator concatenates information from each column for searching adress's long & lat

import pandas as pd
import re

# processor() returns nothing if empty, else returns string

def processor(input):
    pattern = r'\b\w+\b'

    if pd.isnull(input):
        input = ""
    else:
        # Find all matches using the pattern
        matches = re.findall(pattern, input)

        # Convert matches to lowercase and join with a single space in between
        input = ' '.join(match.lower() for match in matches)

    return input

# Iterate over each row in the DataFrame
def address_generator(row):
    column1_value = processor(row['address'])
    column2_value = processor(row['city'])
    column3_value = processor(row['county'])
    column4_value = processor(row['postal'])

    address = f"{column1_value} {column2_value} {column3_value} {column4_value}"
    address = address.strip()
    address = ' '.join(address.split())
    return address

In [34]:
# Creates name from first name and last name columns
def name_generator(row):
    column1_value = processor(row['first_name'])
    column2_value = processor(row['last_name'])

    name = f"{column1_value} {column2_value}"
    name = name.strip()
    name = ' '.join(name.split())
    return name

In [36]:
# Joins all company, name, email and address information together from all processed sources

def complete_stopwords(row):
    column1_value = row['company_name_stopwords']
    column2_value = row['address_name_stopwords']
    column3_value = row['email_stopwords']
    column4_value = row['name_stopwords']

    output = f"{column1_value} {column2_value} {column3_value} {column4_value}"
    output = output.strip()
    output = ' '.join(output.split())
    return output

Organising processed data

In [15]:
# Preprocess Company Name Data
Data["company_key_words"] = Data["company_name"].apply(lambda x: company_processor(str(x)))
Data["address_string"] = Data.apply(lambda row: address_generator(row), axis=1)
Data["email_username"] = Data["email"].apply(lambda x: extract_username(x))
# Data["Co-ordinates"] = Data["address_name_stopwords"].apply(lambda x: find_location(x))
Data["name_stopwords"] = Data.apply(lambda row: preprocess_name(row), axis=1)
Data["all stopwords"] = Data.apply(lambda row: complete_stopwords(row), axis=1)

Similarity using TF-IDF

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=1)
tf_idf_matrix = vectorizer.fit_transform(Data['all stopwords'])

matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 100000, 0.2)

tri_upper_no_diag = triu(matches, k=1)

matches_df = get_matches_SF_brokers(tri_upper_no_diag, Data)

matches_df = pd.merge(matches_df, Data, how='inner', left_on= 'ID' , right_on = 'ID', suffixes = ('', '_1'))
matches_df = pd.merge(matches_df, Data, how='inner', left_on= 'MATCHED_ID' , right_on = 'ID', suffixes = ('', '_MATCHED'))

SELFTIMED: 0.00700688362121582


Second-Layer of checking between matches

In [21]:
# Generate score for matching between fields of same columns

# For names, check simialrity of sound.
from fuzzywuzzy import fuzz

def Similar_Sound_Of_Name(a, b):
    Similarity1 = 0
    if pd.notnull(a) and pd.notnull(b):
        Similarity1 = (fuzz.ratio(a, b))/200
    else:
        Similarity1 = 0.25
    return Similarity1

def discLeven(a, b):
    return abd.DiscountedLevenshtein().sim(a,b)

# For company name, check difference in letters
def company_name_similarity(x, y):
    if pd.notnull(x) and pd.notnull(y):
        return discLeven(x,y)
    else:
        return 0.5

# For email username, check difference in letters
def email_similarity(x, y):
    x.replace(" ", "")
    y.replace(" ", "")
    z = 0
    if pd.notnull(x) and pd.notnull(y):
        z =  discLeven(x,y)
    else:
        z = 0.5
    return z

/Users/v.tiwari/Library/Python/3.9/lib/python/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [24]:
# Give score for matches using process above

matches_df['Name Similarity'] = matches_df.apply(lambda x: (Similar_Sound_Of_Name(x['first_name'],x['first_name_MATCHED']) + Similar_Sound_Of_Name(x['last_name'],x['last_name_MATCHED'])), axis=1)
matches_df['Company Name Similarity'] = matches_df.apply(lambda x: company_name_similarity(x['company_name_stopwords'],x['company_name_stopwords_MATCHED']),axis=1)
# matches_df['Location Closeness'] = matches_df.apply(lambda x: hs.haversine(x['Co-ordinates'],x['Co-ordinates_MATCHED']),axis=1)
matches_df['Email Similarity'] = matches_df.apply(lambda x: email_similarity(x['email_stopwords'],x['email_stopwords_MATCHED']),axis=1)

In [28]:
# Calculate average of similarity and implement a threshold to define matches

import statistics

# matches_df['SIMILARITY'] = matches_df.apply(lambda x: statistics.fmean(x[['Name Similarity','Company Name Similarity','Location Closeness', 'Email Similarity']]),axis=1)
matches_df['SIMILARITY'] = matches_df.apply(lambda x: statistics.fmean(x[['Name Similarity','Company Name Similarity', 'Email Similarity']]),axis=1)
salesforce_Similarity = matches_df.loc[matches_df['SIMILARITY'] >= 0.2]
out = salesforce_Similarity[['ID', 'SIMILARITY', 'MATCHED_ID']]
out.to_csv('SampleData_output.csv',index=False)

                    ID  SIMILARITY          MATCHED_ID
0   0018d000006ytVSCLR    0.793270  0018d000006ytVSAXX
1   0018d000006ytVSALT    0.833333  0018d000006ytVSAXF
2   0018d000006ytVSAWS    0.480321  0018d000006ytVSAXF
3   0018d000006ytVSALT    0.396988  0018d000006ytVSAWS
4   0018d000006ytVSAAV    0.526176  0018d000006ytVSAWS
..                 ...         ...                 ...
77  0018d000006ytVSAVF    1.000000  0018d000006ytVSAWZ
78  0018d000006ytVSAVJ    0.860314  0018d000006ytVSAWY
79  0018d000006ytVSAVP    0.638638  0018d000006ytVSAWT
80  0018d000006ytVSAVV    0.966618  0018d000006ytVSAWX
81  0018d000006ytVSAWH    0.871747  0018d000006ytVSAWW

[78 rows x 3 columns]


# Creating groups of duplicates

In [30]:
# Make a graph, use matches as edges and IDs as Nodes. Give MDM_ID to similar records.

import networkx

sf_brokers = pd.read_csv('SampleData.csv')
sf_matches = pd.read_csv('SampleData_output.csv')
sf_brokers = sf_brokers.replace(np.nan, '', regex=True)
# create all groups 
from networkx import connected_components, draw

nodelist = sf_brokers['ID'].values
edgelist = [(x,y) for x,y in zip(sf_matches['ID'].values, sf_matches['MATCHED_ID'].values) if y != 0]

G = nx.Graph()
G.add_nodes_from(nodelist)
G.add_edges_from(edgelist)
cc = connected_components(G)

tp = []
i = 0
for x in list(cc):
    i += 1
    for r in x:
        tp.append((r, 'SF_BR_'+str(i),i))

df = pd.DataFrame.from_records(tp, columns=['ID', 'MDM_ID','MDM_ID_NUM'])
sf_brokers_match = pd.merge(sf_brokers, df, on='ID')
sf_brokers_match.to_csv('SampleData_Groups.csv', index = False)

sf_brokers_match.sort_values('MDM_ID_NUM', inplace=True, ascending=True)
sf_brokers_match.drop('MDM_ID_NUM', axis=1)
sf_brokers_match.to_excel('SampleData_groups.xlsx', index = False)

# Generate Golden Records

In [112]:
# Lists all non unique Records

data_list = sf_brokers_match['MDM_ID_NUM']
from collections import Counter

counter = Counter(data_list)
non_unique_values = [value for value, count in counter.items() if count > 1]
print(non_unique_values)

[2, 4, 11, 12, 32, 38, 39, 61, 64, 76, 77, 90, 93, 96, 119, 121, 145, 149, 151, 162, 170, 184, 192, 202, 204, 207, 212, 233, 238, 240, 252, 258, 262, 267, 268, 271, 274, 280, 285, 302, 307, 313, 319, 321, 328, 329, 363, 369, 374, 380, 385, 388, 401, 420, 424, 426, 433, 444, 448, 454, 459, 471]


In [113]:
# Under Construction - Ask which records to keep which to remove and so on

Answer = ""
while Answer != "N":
    print(non_unique_values)
    hello = input("here are the non unique ids, which would you like?, if you want to end press N")
    print("here is the data:")
    result = df[df['MDM_ID_NUM'] == Answer]
    print(result)
    print("which values would you like to group together, separate by commas")
    Answer2 = input()
    Answer2 = Answer2.replace(" ", "")
    result_set = list(Answer2.split(','))
    Specific = df.iloc[result_set]


[2, 4, 11, 12, 32, 38, 39, 61, 64, 76, 77, 90, 93, 96, 119, 121, 145, 149, 151, 162, 170, 184, 192, 202, 204, 207, 212, 233, 238, 240, 252, 258, 262, 267, 268, 271, 274, 280, 285, 302, 307, 313, 319, 321, 328, 329, 363, 369, 374, 380, 385, 388, 401, 420, 424, 426, 433, 444, 448, 454, 459, 471]
here is the data:
Empty DataFrame
Columns: [ID, MDM_ID, MDM_ID_NUM]
Index: []
which values would you like to group together, separate by commas


ValueError: invalid literal for int() with base 10: ''